In [1]:
import numpy as np
import pandas as pd   
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import torch
import itertools

import pyro
import pyro.distributions as dist
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal
from pyro.infer import MCMC, NUTS, HMC, SVI, Trace_ELBO
from pyro.optim import Adam, ClippedAdam

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# fix random generator seed (for reproducibility of results)
np.random.seed(42)

# matplotlib options
palette = itertools.cycle(sns.color_palette())
plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 10)

In [2]:
df = pd.read_csv("Data/df_cush.csv")
df = df.dropna()


In [3]:
y = df.DEP_DELAY
dates = df.FL_DATE
X = df.drop(["DEP_DELAY","FL_DATE"],axis=1)
X = X.drop(X.columns[336],axis=1)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# standardize input features
X_train_mean = X_train.mean(axis=0)
X_train_std = X_train.std(axis=0)
X_train = (X_train - X_train_mean) / X_train_std

X_test_mean = X_test.mean(axis=0)
X_test_std = X_test.std(axis=0)
X_test = (X_test - X_test_mean) / X_test_std

# standardize target
y_train_mean = y_train.mean()
y_train_std = y_train.std()
y_train = (y_train - y_train_mean) / y_train_std

y_test_mean = y_test.mean()
y_test_std = y_test.std()
y_test = (y_test - y_test_mean) / y_test_std


In [5]:
print("num train: %d" % len(y_train))
print("num test: %d" % len(y_test))

num train: 1110920
num test: 547171


In [6]:
def compute_error(trues, predicted):
    corr = np.corrcoef(predicted, trues)[0,1]
    mae = np.mean(np.abs(predicted - trues))
    rae = np.sum(np.abs(predicted - trues)) / np.sum(np.abs(trues - np.mean(trues)))
    rmse = np.sqrt(np.mean((predicted - trues)**2))
    r2 = max(0, 1 - np.sum((trues-predicted)**2) / np.sum((trues - np.mean(trues))**2))
    return corr, mae, rae, rmse, r2

REALLY bad model

First a model with TAXI_OUT, DESTINATION and ORIGIN as inpot with same regression coefficients

In [7]:
def model(X, obs=None):
    alpha = pyro.sample("alpha", dist.Normal(0., 1.))                   # Prior for the bias/intercept
    beta  = pyro.sample("beta", dist.Normal(torch.zeros(X.shape[1]), 
                                            torch.ones(X.shape[1])).to_event())    # Priors for the regression coeffcients
    sigma = pyro.sample("sigma", dist.HalfCauchy(5.))                   # Prior for the variance
    
    with pyro.plate("data"):
        y = pyro.sample("y", dist.Normal(alpha + X.matmul(beta), sigma), obs=obs)
        
    return y

In [8]:
# Prepare data for Pyro model
X_train_small = torch.tensor(X_train.to_numpy()[:1000,:]).float()
y_train_small = torch.tensor(y_train.to_numpy()[:1000]).float()

In [9]:
# Run inference in Pyro
nuts_kernel = NUTS(model)
mcmc = MCMC(nuts_kernel, num_samples=1000, warmup_steps=200, num_chains=1)
mcmc.run(X_train_small, y_train_small)

# Show summary of inference results
mcmc.summary()


Sample: 100%|██████████| 1200/1200 [04:08,  4.83it/s, step size=1.02e-01, acc. prob=0.902]



                mean       std    median      5.0%     95.0%     n_eff     r_hat
     alpha      0.04      0.26      0.04     -0.43      0.41    231.49      1.01
   beta[0]      0.03      0.05      0.03     -0.05      0.10    549.60      1.00
   beta[1]      0.08      1.01      0.06     -1.53      1.76    545.96      1.00
   beta[2]     -0.01      0.03     -0.01     -0.05      0.04     47.21      1.06
   beta[3]      0.07      0.96      0.12     -1.39      1.73    775.67      1.00
   beta[4]     -0.00      0.02     -0.01     -0.03      0.02     39.04      1.07
   beta[5]     -0.03      1.03     -0.01     -1.75      1.57   1192.25      1.00
   beta[6]     -0.01      1.00     -0.01     -1.84      1.45   1844.27      1.00
   beta[7]      0.00      1.00     -0.01     -1.72      1.56    688.54      1.00
   beta[8]      0.00      0.04      0.00     -0.06      0.07     46.03      1.07
   beta[9]     -0.02      1.04     -0.07     -1.70      1.67    557.27      1.00
  beta[10]     -0.01      0

In [10]:
# Extract samples from posterior
posterior_samples = mcmc.get_samples()

# Compute predictions
y_hat = np.mean(posterior_samples["alpha"].numpy().T + np.dot(X_test.to_numpy(), posterior_samples["beta"].numpy().T), axis=1)

# Convert back to the original scale
preds = y_hat * y_test_std + y_test_mean
y_true = y_test * y_test_std + y_test_mean

corr, mae, rae, rmse, r2 = compute_error(y_true, preds)
print("CorrCoef: %.3f\nMAE: %.3f\nRMSE: %.3f\nR2: %.3f" % (corr, mae, rmse, r2))


CorrCoef: 0.005
MAE: 25.340
RMSE: 53.288
R2: 0.000


In [11]:
# Try very little subset of data
Xnn = X.iloc[:1000,:]
ynn = y.iloc[:1000]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(Xnn, ynn, test_size=0.33, random_state=42)

In [13]:
########### Try Neural Network model
Xnn = torch.tensor(Xnn.to_numpy()).float()
ynn = torch.tensor(ynn.to_numpy()).float()

In [14]:

class FFNN(torch.nn.Module):
    def __init__(self, n_in, n_hidden, n_out):
        super(FFNN, self).__init__()
        
        # Architecture
        self.in_layer = torch.nn.Linear(n_in, n_hidden)
        self.h_layer = torch.nn.Linear(n_hidden, n_hidden)
        self.out_layer = torch.nn.Linear(n_hidden, n_out)
        
        # Activation functions
        self.tanh = torch.nn.Tanh()
        
    def forward(self, X):
        # Forward pass
        X = self.tanh(self.in_layer(X))
        X = self.tanh(self.h_layer(X))
        X = self.out_layer(X)
        
        return X

In [15]:
def nnet_model(X, y=None):
    # Initialize the neural network from PyTorch 
    torch_model = FFNN(n_in=X.shape[1], n_hidden=4, n_out=1) 
    
    # Convert the PyTorch neural net into a Pyro model with priors
    priors = {} # Priors for the neural model
    for name, par in torch_model.named_parameters():     # Loop over all neural network parameters
        priors[name] = dist.Normal(torch.zeros(*par.shape), torch.ones(*par.shape)).to_event() # Each parameter has a N(0, 1) prior
    
    bayesian_model = pyro.random_module('bayesian_model', torch_model, priors) # Make this model and these priors a Pyro model
    sampled_model = bayesian_model()                                           # Initialize the model
    
    # The generative process
    with pyro.plate("observations"):
        prediction_mean = sampled_model(X).squeeze(-1) # Feed-forward the design matrix X through the neural network
        y = pyro.sample("obs", dist.Normal(prediction_mean, 0.1), obs=y)
        
    return y

In [16]:
# Define guide function
guide = AutoDiagonalNormal(nnet_model)

# Reset parameter values
pyro.clear_param_store()

In [17]:
# Define the number of optimization steps
n_steps = 10000

# Setup the optimizer
adam_params = {"lr": 0.01}
optimizer = Adam(adam_params)

# Setup the inference algorithm
elbo = Trace_ELBO(num_particles=1)
svi = SVI(nnet_model, guide, optimizer, loss=elbo)

# Do gradient steps
for step in range(n_steps):
    elbo = svi.step(Xnn, ynn)
    if step % 500 == 0:
        print("[%d] ELBO: %.1f" % (step, elbo))

c:\Users\chris\anaconda3\lib\site-packages\pyro\primitives.py:491: FutureWarning: The `random_module` primitive is deprecated, and will be removed in a future release. Use `pyro.nn.Module` to create Bayesian modules from `torch.nn.Module` instances.
  warnings.warn(


[0] ELBO: 82530294.2
[500] ELBO: 70993898.2
[1000] ELBO: 67977564.9
[1500] ELBO: 64977375.5
[2000] ELBO: 61940984.1
[2500] ELBO: 59908025.4
[3000] ELBO: 58302806.4
[3500] ELBO: 56723692.4
[4000] ELBO: 55365883.0
[4500] ELBO: 54582598.2
[5000] ELBO: 53912903.9
[5500] ELBO: 53387126.5
[6000] ELBO: 52998944.7
[6500] ELBO: 52776861.7
[7000] ELBO: 52427180.9
[7500] ELBO: 52168328.4
[8000] ELBO: 52001029.2
[8500] ELBO: 51913061.6
[9000] ELBO: 51767192.4
[9500] ELBO: 51681162.7


In [19]:
# Prepare test data for Pyro
X_test = torch.tensor(X_test.to_numpy()).float()

In [20]:
from pyro.infer import Predictive

# Make predictions for test set
predictive = Predictive(nnet_model, guide=guide, num_samples=1000,
                        return_sites=("obs", "_RETURN"))
samples = predictive(X_test)

c:\Users\chris\anaconda3\lib\site-packages\pyro\primitives.py:491: FutureWarning: The `random_module` primitive is deprecated, and will be removed in a future release. Use `pyro.nn.Module` to create Bayesian modules from `torch.nn.Module` instances.
  warnings.warn(


In [21]:
def nnet_interpretable_model(X, y=None):
    # Initialize the neural network from PyTorch 
    torch_model = FFNN(n_in=X.shape[1]-1, n_hidden=4, n_out=1) 
    
    # Convert the PyTorch neural net into a Pyro model with priors
    priors = {} # Priors for the neural model
    for name, par in torch_model.named_parameters():     # Loop over all neural network parameters
        priors[name] = dist.Normal(torch.zeros(*par.shape), torch.ones(*par.shape)).to_event() # Each parameter has a N(0, 1) prior
    
    bayesian_model = pyro.random_module('bayesian_model', torch_model, priors) # Make this model and these priors a Pyro model
    sampled_model = bayesian_model()                                           # Initialize the model
    
    # Linear model priors
    beta_lin = pyro.sample("beta", dist.Normal(0, 1))
    
    # The generative process
    with pyro.plate("observations"):
        linear_out = X[:,0]*beta_lin
        nn_out = sampled_model(X[:,1:]).squeeze(-1) # Feed-forward the design matrix X through the neural network
        y = pyro.sample("obs", dist.Normal(linear_out+nn_out, 0.1), obs=y)
        
    return y

In [22]:
# Define guide function
guide = AutoDiagonalNormal(nnet_interpretable_model)

# Reset parameter values
pyro.clear_param_store()

# Define the number of optimization steps
n_steps = 10000

# Setup the optimizer
adam_params = {"lr": 0.01}
optimizer = ClippedAdam(adam_params)

# Setup the inference algorithm
elbo = Trace_ELBO(num_particles=1)
svi = SVI(nnet_interpretable_model, guide, optimizer, loss=elbo)

# Do gradient steps
for step in range(n_steps):
    elbo = svi.step(Xnn, ynn)
    if step % 500 == 0:
        print("[%d] ELBO: %.1f" % (step, elbo))

[0] ELBO: 79205579.7
[500] ELBO: 70457553.3
[1000] ELBO: 69417837.9
[1500] ELBO: 68921620.3
[2000] ELBO: 68660881.3
[2500] ELBO: 68553536.2
[3000] ELBO: 68466813.9
[3500] ELBO: 68516454.8
[4000] ELBO: 68378551.5
[4500] ELBO: 68383309.5
[5000] ELBO: 68341340.2
[5500] ELBO: 68337654.6
[6000] ELBO: 68359000.4
[6500] ELBO: 68324554.3
[7000] ELBO: 68329634.5
[7500] ELBO: 68352366.0
[8000] ELBO: 68318056.4
[8500] ELBO: 68318660.7
[9000] ELBO: 68339623.0
[9500] ELBO: 68323272.8


In [23]:
from pyro.infer import Predictive

# Make predictions for test set
predictive = Predictive(nnet_interpretable_model, guide=guide, num_samples=1000,
                        return_sites=("obs", "_RETURN"))
samples = predictive(X_test)

In [25]:
y_pred = samples["obs"].mean(axis=0).detach().numpy()

corr, mae, rae, rmse, r2 = compute_error(y_test, y_pred)
print("CorrCoef: %.3f\nMAE: %.3f\nRMSE: %.3f\nR2: %.3f" % (corr, mae, rmse, r2))

CorrCoef: 0.253
MAE: 21.096
RMSE: 40.379
R2: 0.061


In [26]:
y_pred

array([25.944462  , 21.86322   , 20.650864  ,  0.74684227,  7.2856917 ,
       26.555796  , -4.4102335 ,  7.89875   , 26.56263   , -2.0773768 ,
        6.980639  ,  1.5553894 ,  5.4555364 ,  5.0229826 , 24.907972  ,
       27.777182  , 17.622673  , 24.62341   , -1.7299864 , 22.799974  ,
       20.951872  , -0.5423774 ,  5.0209975 , -2.041235  , 25.952087  ,
        3.2022843 , 25.3463    ,  5.556955  ,  0.2402338 , 22.494467  ,
       -0.23819456, -5.291641  ,  7.902377  , 27.774513  , 13.914909  ,
       25.134142  ,  6.9894147 , 11.567175  , 24.481276  , 17.973888  ,
        5.5174656 ,  9.114879  ,  6.379212  , 11.887705  ,  7.2909102 ,
        5.217951  ,  5.243947  ,  6.0707335 ,  9.1124935 , 25.541658  ,
       27.161118  , 20.362133  ,  4.2685227 ,  6.4045873 , 12.783459  ,
        3.6597235 , 10.359882  ,  0.7501983 ,  7.0532994 , 23.406776  ,
       29.376917  , 11.5654545 ,  4.6065245 , 26.142202  ,  4.877506  ,
        5.026278  ,  1.2817012 , 25.535048  , 12.490388  ,  5.24